In [ ]:
import scanpy as sc
import pandas as pd
import os
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests  
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
def calculate_celltype_proportions(adata, group_column='Group', sample_column='sample', celltype_column='celltype_level3'):
    """
    计算每个样本中每个细胞类型的比例，并删除所有列为 NaN 的行。
    
    参数：
    - adata: AnnData 对象
    - group_column: 分组列的名称（例如 'group'，表示不同的组别）
    - sample_column: 样本列的名称（例如 'sample'，表示每个样本的名称）
    - celltype_column: 细胞类型列的名称（例如 'majority_voting'，表示细胞类型）
    - group_mapping: 映射字典，用于将样本归类为不同的组
    
    返回：
    - proportions: 细胞类型在各个样本中的比例，删除全是 NaN 的行后的数据
    """
    # 计算每个样本中每个细胞类型的比例
    cell_counts = adata.obs.groupby([group_column, sample_column, celltype_column]).size().unstack(fill_value=0)
    
    # 计算每个样本的总细胞数
    total_counts = cell_counts.sum(axis=1)
    
    # 计算每个细胞类型的比例
    proportions = cell_counts.div(total_counts, axis=0)
    
    # 删除全是 NaN 的行
    proportions_cleaned = proportions.dropna(how='all')
    
    return proportions_cleaned

In [ ]:
def run_wilcoxon_for_proportions(proportions, group_column='Group'):
    """
    对每个细胞类型进行 Wilcoxon 检验，比较两个组之间的比例差异（依据行索引中的 'group' 分组）。

    参数：
    - proportions: 细胞类型比例数据 (行索引包含 'group' 和 'sample'，列为细胞类型)
    - group_column: 用于分组的列名（默认是 'group'，在行索引中包含 'N' 和 'S'）

    返回：
    - wilcoxon_results_df: 包含每个细胞类型检验结果的数据框
    """
    # 初始化结果字典
    wilcoxon_results = []

    # 获取所有细胞类型（假设细胞类型是列名）
    celltypes = proportions.columns

    # 对每个细胞类型进行 Wilcoxon 检验
    for celltype in celltypes:
        group1_proportions = proportions.loc[proportions.index.get_level_values(group_column) == 'Control', celltype]
        group2_proportions = proportions.loc[proportions.index.get_level_values(group_column) == 'Treatment', celltype]

        # 确保两组都有数据
        if len(group1_proportions) > 0 and len(group2_proportions) > 0:
            # 执行 Wilcoxon 检验
            stat, p_value = mannwhitneyu(group1_proportions, group2_proportions)

            # 存储结果
            wilcoxon_results.append({
                'celltype': celltype,
                'group1': 'Control',
                'group2': 'Treatment',
                'statistic': stat,
                'p_value': p_value
            })

    # 转换为 DataFrame
    wilcoxon_results_df = pd.DataFrame(wilcoxon_results)

    # 使用 Benjamini-Hochberg 方法调整 p 值
    _, adj_p_values, _, _ = multipletests(wilcoxon_results_df['p_value'], method='fdr_bh')

    # 添加调整后的 p 值
    wilcoxon_results_df['adj_p_value'] = adj_p_values

    return wilcoxon_results_df

In [ ]:
# 计算每个细胞类型的比例
proportions = calculate_celltype_proportions(adata,sample_column='sample', celltype_column='celltype_level3')
print(proportions.head())
# 使用Wilcoxon检验比较两个组（N和S）的细胞类型比例
wilcoxon_results = run_wilcoxon_for_proportions(proportions, group_column='Group')

# 输出包含p_value和adj_p_value的结果
print(wilcoxon_results)
celltype=wilcoxon_results['celltype']

In [ ]:
def plot_significant_celltypes_with_stats(proportions, wilcoxon_results, significant_celltypes, sample_column='sample', group_column='group'):
    """
    绘制显著性细胞类型的散点图，并添加显著性星号。
    
    参数：
    - proportions: 细胞类型比例数据框
    - wilcoxon_results: Wilcoxon 检验结果，包含 p 值
    - significant_celltypes: 显著性细胞类型（p值 < 0.05）
    - sample_column: 样本列名
    - group_column: 分组列名
    """
    # 仅选择显著的细胞类型数据
    sig_data = proportions[significant_celltypes]
    
    # 重设索引，使 sample 和 group 列成为列
    sig_data = sig_data.reset_index()
    
    # 现在， sig_data 应该包含 'sample' 和 'group' 列，可以直接使用 melt 转换为长格式
    melted_data = sig_data.melt(id_vars=[sample_column, group_column], value_vars=significant_celltypes,
                                var_name='celltype', value_name='proportion')
    
    # 设置图表的颜色和样式
    plt.figure(figsize=(6, 6))  # 图表尺寸可以调整

    # 绘制每个显著细胞类型的散点图
    for i, celltype in enumerate(significant_celltypes):
        # 获取每个细胞类型的数据
        celltype_data = melted_data[melted_data['celltype'] == celltype]
        
        # 绘制散点图
        ax = sns.stripplot(data=celltype_data, x='celltype', y='proportion', hue=group_column, dodge=True, jitter=True, palette="Set2", alpha=0.6, size=6)
        
        # 计算 p 值并添加星号
        p_value = wilcoxon_results[wilcoxon_results['celltype'] == celltype]['p_value'].values[0]
                # 星号的位置
        star_y = celltype_data['proportion'].max() + 0.05  # 星号在最大值上方 0.05 的位置
        p_value_y = star_y + 0.02  # p 值在星号上方 0.05 的位置
        if p_value < 0.001:
            ax.text(i, celltype_data['proportion'].max() , '***', ha='center', va='bottom', fontsize=12)
        elif p_value < 0.01:
            ax.text(i, celltype_data['proportion'].max() , '**', ha='center', va='bottom', fontsize=12)
        elif p_value < 0.05:
            ax.text(i, celltype_data['proportion'].max() ,'*', ha='center', va='bottom', fontsize=12)
        else:
            ax.text(i, celltype_data['proportion'].max() , f'p={p_value:.3f}', ha='center', va='bottom', fontsize=10)

    # 设置图表标题和标签
    ax.set_title('Significant Celltype Proportions with P-values', fontsize=16)
    ax.set_xlabel('Cell Types', fontsize=14)
    ax.set_ylabel('Proportion', fontsize=14)
    
    # 设置 x 轴的标签旋转
    plt.xticks(rotation=45)
    
    # 手动添加图例，只显示一次 "N" 和 "S"
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles[:2], labels[:2], title=group_column, loc='upper right')
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_significant_celltypes_with_stats(proportions, wilcoxon_results,celltype, sample_column='sample', group_column='Group')